インストール類(pyworld,pydudははf０のため、ffmpeg,spleeterは音源分離のため)

In [ ]:
pip install pyworld

     |████████████████████████████████| 212 kB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyworld: filename=pyworld-0.3.0-cp37-cp37m-linux_x86_64.whl size=609480 sha256=caf7fab04aabe4a953dae32f577d0221f3627de11308b4e4e0436672be998400
  Stored in directory: /root/.cache/pip/wheels/e7/7c/11/c775fffa0e1e7b05a6604b4323408a77f80fb4ab304d96b5c6
Successfully built pyworld


In [ ]:
pip install pydub

In [ ]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
pip install spleeter

     |████████████████████████████████| 51 kB 4.7 MB/s 
     |████████████████████████████████| 25.3 MB 68.6 MB/s 
     |████████████████████████████████| 183 kB 62.7 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
     |████████████████████████████████| 454.3 MB 17 kB/s 
     |████████████████████████████████| 4.0 MB 49.7 MB/s 
     |████████████████████████████████| 462 kB 52.3 MB/s 
     |████████████████████████████████| 1.2 MB 43.6 MB/s 
     |████████████████████████████████| 58 kB 7.2 MB/s 
     |████████████████████████████████| 57 kB 6.3 MB/s 
     |████████████████████████████████| 54 kB 3.8 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 3.3 MB 73.5 MB/s 
     |████████████████████████████████| 3.4 MB 33.2 MB/s 
     |████████████████████████████████| 3.4 MB 51.3 MB/s 
     |████████████████████████████████| 97 kB 8.0 MB/s 
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201395 sha2

In [ ]:
#多分いらない
#pip install requests

*音源分離*

実行の仕方：!spleeter separate -o 保存先のフォルダのパス 分離ファイルのパス

パスを変数に格納しての使用は不可

In [ ]:
!ls

!spleeter separate -o /content/drive/MyDrive/project4/music_file/ /content/drive/MyDrive/project4/music_file/test/teacher_akapera_1m.wav

drive  sample_data
INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz
INFO:spleeter:Validating archive checksum
INFO:spleeter:Extracting downloaded 2stems archive
INFO:spleeter:2stems model file(s) extracted
INFO:spleeter:File /content/drive/MyDrive/project4/music_file/teacher_akapera_1m/vocals.wav written succesfully
INFO:spleeter:File /content/drive/MyDrive/project4/music_file/teacher_akapera_1m/accompaniment.wav written succesfully


**システムの使い方**：

1.インストール類をインストールする

2.音源が伴奏ありなら、パスを指定し音源分離する

3.本来は単位時間、音源と検索音源のパスを入力して、システムを実行

モジュールのimportで、名前を不用意に変えると、"no module named <モジュール名>"というエラーが出る可能性がある。

In [1]:
#モジュールのためのライブラリ
import importlib
import sys
sys.path.append('/content/drive/MyDrive/project4/notebook')

#モジュールのインポート
import preparation
import f0
import compare_f0
import peak_frec
import compare_peak
import candidate

#時間計測のライブラリ
import time

#ファイル、フォルダ削除のライブラリ
import shutil
import os

import numpy as np


importlib.reload(preparation)
importlib.reload(f0)
importlib.reload(compare_f0)
importlib.reload(peak_frec)
importlib.reload(compare_peak)
importlib.reload(candidate)

ModuleNotFoundError: ignored

In [ ]:
class Smanager:

  def __init__(self):
    #music_path = input("音源のパスを入力してくだい")
    #humming_path = input("検索音源のパスを入力してください")
    #unit_time = input("単位時間を入力してください")
    
    music_path = '/content/drive/MyDrive/project4/music_file/azakawa.wav'
    humming_path = '/content/drive/MyDrive/project4/music_file/azakawa1.wav'
    unit_time = 0.1
    
    #下準備(音源ファイルをモノラルにし、単位時間に分割)
    prepa = preparation.Prepare(music_path, humming_path, unit_time)

    #本体
    self.execution(music_path, humming_path, unit_time)


  def execution(self, music_path, humming_path, unit_time):
    #実行時間計測開始
    self.start_time = time.perf_counter()


    #f0, f0_compareクラス
    music_path = music_path[:-4]+'_mono.wav'
    humming_path = humming_path[:-4]+'_mono.wav'
    
    com = compare_f0.CompareF0(music_path, humming_path, unit_time)

    #if self.judge_diffkey(com.same_song) == False:
      #return
    """
    if com.same_song == 0:
      print("違う曲の可能性が高いです。")
      #実行時間計測終了
      execution_time = time.perf_counter() - start_time
      print(execution_time)
      return
    """


    #peak_frecクラス
    #split_music_path = self.prepa.data_min
    #split_hum_path = self.prepa.test_min

    split_music_path = "/content/drive/MyDrive/data_output"
    split_hum_path = "/content/drive/MyDrive/test_output"

    mpeak = peak_frec.Peak(split_music_path)
    hpeak = peak_frec.Peak(split_hum_path)


    #Compare_peakクラス
    com_peak = compare_peak.Compare_peak(mpeak.peak_list, hpeak.peak_list)
    print("音の差の最小値の数", np.count_nonzero(com_peak.same_score == com_peak.same_score.min()))
    print("音の変化量の差の最小値の数", np.count_nonzero(com_peak.score == com_peak.score.min()))


    #Candidateクラス
    candi = candidate.Candidate(unit_time, com.test_time)
    #candi.add_score(com.norm_gosa)
    candi.add_score(com_peak.score)

    leng = length(unit_time)
    start, end = candi.last()
    start = round(start, leng)
    end = round(end,leng)
    print(start, "秒から", end, "です")
    
    
    #実行時間計測終了
    self.execution_time = time.perf_counter() - self.start_time
    print(self.execution_time)
    
    shutil.rmtree(split_music_path)
    shutil.rmtree(split_hum_path)
    os.remove(music_path)
    os.remove(humming_path)


  def judge_diffkey(self, judge):
    if judge == 0:
      print("違う曲の可能性が高いです。")
      self.execution_time = time.perf_counter() - self.start_time
      print("実行時間：", self.execution_time)
      return False
    return True


    def length(self, time):
      if type(time) is int:
        return 0
      else:
        time = str(time)
        s_i, s_d = time.split('.')
        return len(s_d)

In [ ]:
sm = Smanager()

**旧システム**(下準備)

In [ ]:
#モジュールのためのライブラリ
import importlib
import sys
sys.path.append('/content/drive/MyDrive/project4/notebook')

#モジュールのインポート
import preparation
import f0
import compare_f0
import peak_frec
import compare_peak
import candidate

#時間計測のライブラリ
import time

#ファイル、フォルダ削除のライブラリ
import shutil
import os

importlib.reload(preparation)

#unit_time = input("単位時間を入力してください")

#preparationクラス
music_path = '/content/drive/MyDrive/project4/music_file/test/teacher_akapera_1m.wav'
humming_path = '/content/drive/MyDrive/project4/music_file/test/sing_5s.wav'

unit_time = 0.1

prepa = preparation.Prepare(music_path, humming_path, unit_time)


In [ ]:
importlib.reload(preparation)
importlib.reload(f0)
importlib.reload(compare_f0)
importlib.reload(peak_frec)
importlib.reload(compare_peak)
importlib.reload(candidate)

<module 'candidate' from '/content/drive/MyDrive/project4/notebook/candidate.py'>

In [ ]:
import numpy as np

def length(time):
  if type(time) is int:
    return 0
  else:
    time = str(time)
    s_i, s_d = time.split('.')
    return len(s_d)


def smanager(music_path, humming_path, unit_time):

  music_path = music_path[:-4]+'_mono.wav'
  split_music_path = ('/content/drive/MyDrive/data_output/')
  split_hum_path = ('/content/drive/MyDrive/test_output/')
  mpeak = peak_frec.Peak(split_music_path)


  #実行時間計測開始
  start_time = time.perf_counter()
  
  
  #f0, f0_compareクラス
  humming_path = humming_path[:-4]+'_mono.wav'
  
  com = compare_f0.CompareF0(music_path, humming_path, unit_time)


  #peak
  hpeak = peak_frec.Peak(split_hum_path)
    
    
  #Compare_peakクラス
  com_peak = compare_peak.Compare_peak(mpeak.peak_list, hpeak.peak_list)
  print("音の差の最小値の数", np.count_nonzero(com_peak.same_score == com_peak.same_score.min()))
  print("音の変化量の差の最小値の数", np.count_nonzero(com_peak.score == com_peak.score.min()))

    
    
  #Candidateクラス
  candi = candidate.Candidate(unit_time, com.test_time)
  #candi.add_score(com.norm_gosa)
  #candi.add_score(com_peak.same_score)
  candi.add_score(com_peak.score)

  start, end = candi.last()
  print(start, "秒から", end, "です")
    
    
  #実行時間計測終了
  execution_time = time.perf_counter() - start_time
  print(execution_time)

  shutil.rmtree('/content/drive/MyDrive/data_output')
  shutil.rmtree('/content/drive/MyDrive/test_output')
  os.remove(music_path)
  os.remove(humming_path)


In [ ]:
smanager(music_path, humming_path, unit_time)

音の差の最小値 130
音の変化量の差の最小値 74
音の差の最小値の数 1
音の変化量の差の最小値の数 1
(array([], dtype=int64),)
29.0 秒から 34.000249433106575 です
5.156677653000315


旧システム本体

In [ ]:
import numpy as np

def length(time):
  if type(time) is int:
    return 0
  else:
    time = str(time)
    s_i, s_d = time.split('.')
    return len(s_d)


def smanager(music_path, humming_path, unit_time):
  #実行時間計測開始
  start_time = time.perf_counter()
  
  
  #f0, f0_compareクラス
  music_path = music_path[:-4]+'_mono.wav'
  humming_path = humming_path[:-4]+'_mono.wav'
  
  com = compare_f0.CompareF0(music_path, humming_path, unit_time)
  
  """
  if com.same_song == 0:
    print("違う曲の可能性が高いです。")
    #実行時間計測終了
    execution_time = time.perf_counter() - start_time
    print(execution_time)
    return
  """
    
  #peak
  split_music_path = ('/content/drive/MyDrive/data_output/')
  split_hum_path = ('/content/drive/MyDrive/test_output/')
    
  mpeak = peak_frec.Peak(split_music_path)
  hpeak = peak_frec.Peak(split_hum_path)

  #print(mpeak)
  #print(hpeak)
    
    
  #Compare_peakクラス
  com_peak = compare_peak.Compare_peak(mpeak.peak_list, hpeak.peak_list)
  print("音の差の最小値の数", np.count_nonzero(com_peak.same_score == com_peak.same_score.min()))
  print("音の変化量の差の最小値の数", np.count_nonzero(com_peak.score == com_peak.score.min()))

    
    
  #Candidateクラス
  candi = candidate.Candidate(unit_time, com.test_time)
  candi.add_score(com.norm_gosa)
  candi.add_score(com_peak.same_score)

  #leng = length(unit_time)
  #print(leng)
  start, end = candi.last()
  #start = round(start, leng+1)
  #end = round(end,leng+1)
  print(start, "秒から", end, "です")
    
    
  #実行時間計測終了
  execution_time = time.perf_counter() - start_time
  print(execution_time)

  shutil.rmtree('/content/drive/MyDrive/data_output')
  shutil.rmtree('/content/drive/MyDrive/test_output')
  os.remove(music_path)
  os.remove(humming_path)


In [ ]:
smanager(music_path, humming_path, unit_time)

音の差の最小値 109
音の変化量の差の最小値 65
音の差の最小値の数 1
音の変化量の差の最小値の数 1
34.4 秒から 39.40024943310657 です
82.02273318100015


In [ ]:
def length(time):
  if type(time) is int:
    return 0
  else:
    time = str(time)
    s_i, s_d = time.split('.')
    return len(s_d)


cut_time = 0.1
b = length(cut_time)
print(b)

a = 408 * 0.10
print(a)

a = round(a, b)
print(a)


1
40.800000000000004
41.0
